In [1]:
# %% [markdown]
# # Regressão Ordinal — Práticas Agrícolas x Nível de Degradação
# Avalia o efeito conjunto de práticas sustentáveis ou convencionais
# sobre a variável ordinal "degradação percebida".

# %%
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel

# %% [markdown]
# ## 1. Carregar e preparar os dados
# Ajuste o arquivo conforme o conjunto desejado (Convencional ou Sustentável)

# %%
# Caminho do arquivo (use o dataset que já foi codificado anteriormente)
file_path = r"C:/Users/Valentine/Artigo_Mapitos/data_clean/07_Codificado/07_Convencionais_Degradacao_Linha.csv"
df = pd.read_csv(file_path)

# Verificar as colunas
print("Dimensões:", df.shape)
df.head()

# %% [markdown]
# ## 2. Identificar colunas relevantes
# - As colunas de práticas começam com "pratica_"
# - A variável dependente é "degradacao_ord"

# %%
col_praticas = [c for c in df.columns if c.startswith("pratica_")]
y_col = "degradacao_ord"

print(f"Colunas de práticas ({len(col_praticas)}):", col_praticas)
print("Variável dependente:", y_col)

# %% [markdown]
# ## 3. Remover linhas com valores ausentes
# Evita erros no modelo

# %%
df = df.dropna(subset=[y_col])
X = df[col_praticas]
y = df[y_col]

print("Tamanho final do dataset:", X.shape)

# %% [markdown]
# ## 4. Definir e ajustar o modelo ordinal
# A função `OrderedModel` estima uma regressão logística ordinal (logit link).

# %%
# Criar modelo
model = OrderedModel(
    endog=y,
    exog=X,
    distr="logit"  # pode usar "probit" também
)

# Ajustar modelo
res = model.fit(method="bfgs", disp=False)

# %%
# Resumo dos resultados
print(res.summary())

# %% [markdown]
# ## 5. Interpretar resultados
# Coeficientes negativos → reduzem a chance de degradação mais alta
# Coeficientes positivos → aumentam a chance de degradação mais alta

# %%
# Tabela organizada
coef = pd.DataFrame({
    "Prática": X.columns.str.replace("pratica_", "").str.replace("_", " ").str.title(),
    "Coeficiente": res.params,
    "p-valor": res.pvalues
}).reset_index(drop=True)

# Adicionar interpretação
coef["Efeito"] = np.where(coef["Coeficiente"] < 0, "Reduz degradação", "Aumenta degradação")

# Ordenar por p-valor
coef = coef.sort_values(by="p-valor")

# Exibir principais resultados
coef.head(10)

# %%
# Salvar resultados
output_path = r"C:/Users/Valentine/Artigo_Mapitos/data_analysis/regressao_ordinal_Conv.csv"
coef.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ Resultados exportados para:", output_path)


Dimensões: (975, 17)
Colunas de práticas (10): ['pratica_aração', 'pratica_controle biológico de pragas e doenças.', 'pratica_gradagem', 'pratica_monocultura', 'pratica_não usa', 'pratica_pastagem nativa', 'pratica_plantio convencional', 'pratica_queimadas', 'pratica_roça de toco', 'pratica_uso de agrotóxicos']
Variável dependente: degradacao_ord
Tamanho final do dataset: (975, 10)


ValueError: There should not be a constant in the model

In [2]:
# Ajuste robusto para OrderedModel (corrige erro de "constant" e sanitiza nomes)
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel

# caminho do arquivo (ajuste se necessário)
file_path = r"C:/Users/Valentine/Artigo_Mapitos/data_clean/07_Codificado/07_Convencionais_Degradacao_Linha.csv"
df = pd.read_csv(file_path)

# garantir que a variável dependente exista e seja inteira ordinal
y_col = "degradacao_ord"
if y_col not in df.columns:
    raise ValueError(f"Coluna {y_col} não encontrada no dataframe.")

df = df.dropna(subset=[y_col]).copy()
df[y_col] = df[y_col].astype(int)

# selecionar colunas de prática (prefixo 'pratica_')
col_praticas = [c for c in df.columns if c.startswith("pratica_")]
print("Práticas detectadas:", col_praticas)

# 1) sanitizar nomes das colunas (remover espaços, acentos e pontuação)
def clean_colname(s):
    s = str(s).strip().lower()
    s = s.replace(" ", "_")
    s = s.replace("-", "_")
    s = s.replace(".", "")
    s = s.replace("/", "_")
    # remover caracteres não-ascii básicos
    import re, unicodedata
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s = re.sub(r"[^0-9a-z_]", "", s)
    return s

df = df.rename(columns={c: clean_colname(c) for c in df.columns})
# atualizar listas após limpeza
col_praticas = [c for c in df.columns if c.startswith("pratica_")]
print("Práticas após limpeza:", col_praticas)

# 2) remover colunas constantes (todas iguais)
const_cols = [c for c in col_praticas if df[c].nunique() == 1]
if const_cols:
    print("Removendo colunas constantes (único valor):", const_cols)
    df = df.drop(columns=const_cols)
    col_praticas = [c for c in col_praticas if c not in const_cols]

# 3) opcional: remover dummy referência ('pratica_nao_usa' ou similar)
ref_candidates = [c for c in col_praticas if "nao" in c or "nao_usa" in c or "naousa" in c]
if ref_candidates:
    print("Removendo dummy de referência (para evitar redundância):", ref_candidates)
    df = df.drop(columns=ref_candidates)
    col_praticas = [c for c in col_praticas if c not in ref_candidates]

# 4) preparar X e y
X = df[col_praticas].astype(float)
y = df[y_col]

print("Tamanho do dataset para modelagem:", X.shape)

# 5) verificar se há colinearidade perfeita (matriz X'X singular)
# (checar determinante ou rank)
if np.linalg.matrix_rank(X.values) < X.shape[1]:
    print("== Atenção: colinearidade detectada entre colunas de X. Tentando remover colunas lineares correlacionadas ==")
    # estratégia simples: remover uma coluna com correlação 1 com outra
    corr = X.corr().abs()
    to_drop = set()
    for i in corr.columns:
        for j in corr.columns:
            if i != j and corr.loc[i,j] > 0.999999:
                # escolha j para remover
                to_drop.add(j)
    if to_drop:
        print("Removendo colunas quase-idênticas:", to_drop)
        X = X.drop(columns=list(to_drop))
        col_praticas = [c for c in col_praticas if c not in to_drop]
        print("Novas práticas:", col_praticas)

# 6) ajustar modelo ordinal (sem constante)
try:
    model = OrderedModel(endog=y, exog=X, distr="logit")
    res = model.fit(method="bfgs", maxiter=200, disp=True)
    print(res.summary())
except Exception as e:
    print("Erro ao ajustar o modelo ordinal:", str(e))
    print("Sugestões: 1) aumentar maxiter, 2) reduzir colunas por alto VIF, 3) testar 'probit' ou usar subset dos dados.")


Práticas detectadas: ['pratica_aração', 'pratica_controle biológico de pragas e doenças.', 'pratica_gradagem', 'pratica_monocultura', 'pratica_não usa', 'pratica_pastagem nativa', 'pratica_plantio convencional', 'pratica_queimadas', 'pratica_roça de toco', 'pratica_uso de agrotóxicos']
Práticas após limpeza: ['pratica_aracao', 'pratica_controle_biologico_de_pragas_e_doencas', 'pratica_gradagem', 'pratica_monocultura', 'pratica_nao_usa', 'pratica_pastagem_nativa', 'pratica_plantio_convencional', 'pratica_queimadas', 'pratica_roca_de_toco', 'pratica_uso_de_agrotoxicos']
Removendo dummy de referência (para evitar redundância): ['pratica_nao_usa']
Tamanho do dataset para modelagem: (975, 9)
Optimization terminated successfully.
         Current function value: 1.205295
         Iterations: 62
         Function evaluations: 64
         Gradient evaluations: 64
                             OrderedModel Results                             
Dep. Variable:         degradacao_ord   Log-Likelihoo